In [1]:
import icalendar
import pytz, zoneinfo, dateutil.tz  # timezone libraries
import datetime, icalendar
import pyaml
import pandas as pd 
import numpy as np
import yaml
from datetime import datetime, timedelta
import re
from ics import Calendar, Event

In [2]:



def convert_datestring (datestring):
    format = "%m/%d/%Y"
    date_object = datetime.strptime(datestring, format)
    return date_object


def get_start_end_dates(block_id, my_dict):
    start, end = (my_dict['block_dates'][block_id].split())
    start = convert_datestring(start)
    end = convert_datestring(end)
    return start, end


def date_lists(start_date, end_date):
    current_date = start_date
    dates = []
    while current_date <= end_date:
        dates.append(current_date)
        current_date += timedelta(days=1)
    return dates
    
    
def get_specific_rotation_schedule(block_id, rotation, my_dict):
    rotation = re.sub('\s', '', rotation)
    

        
    try:
        rotation, individual = rotation.split('-')
        individual = individual[0] # gets the first character of an individual (A, B, C, D, E, F etc.)
    except:
        rotation = rotation
        individual = 'A'
        
    if re.search('Janeway|Barker|Longcope|Thayer', rotation):
        rotation_category = 'O'    
    elif re.search('CJ|Polk|MEG', rotation):
        rotation_category = 'Subspecialty'    
        
    elif re.search('PCCU|Addiction', rotation):
        # these rotation have MTWTF rotation with weekends off
        rotation_category = 'Ambulatory'
    else:
        rotation_category = rotation
        
    for schedule_type, schedule in my_dict[rotation_category].items():
        if block_id in schedule['blocks']:
            return schedule[individual].split()
        
        
        
def block_dates (block_id, my_dict):
    """get start and end date for each block by going through the master yaml """
    start_date, end_date = (get_start_end_dates(block_id, my_dict))
    block_duration = date_lists(start_date, end_date)    
    return block_duration
 

def add_date_postcall_into_next_block(call_cycle, block_days):
    print(call_cycle[-1])
    if re.search(r'^\*', call_cycle[-1]):
        # Step 1: Get the last date in the list
        last_date = block_days[-1]

        # Step 2: Calculate the next date (assuming consecutive days, so add 1 day)
        next_date = last_date + timedelta(days=1)

        # Step 3: Add the new date to the list
        block_days.append(next_date)
    return block_days
    



def oncology_schedule(rotation, date):
    if rotation== 'Subspecialty': # if subspecialty rotation, default is 'nights'
        schedule = 'NIGHT'
    else:
        schedule = 'DAY'
    
    if date.weekday() == 6 and rotation in ['Solids', 'MTL', 'Leuks']: #off day is Sunday
        schedule = 'OFF'
    elif date.weekday() == 5 and rotation in ['Ambulatory', 'UCM', 'AddictionMedicine', 'Psych']: #ambo off day is Saturday and Sunday
        schedule = 'OFF'   
    elif date.weekday() == 6 and rotation in ['Ambulatory', 'UCM', 'AddictionMedicine', 'Psych']: #ambo off day is Saturday and Sunday
        schedule = 'OFF'
    elif date.weekday() == 6 and rotation == 'Subspecialty': # subspecialty intern off on Sunday
        schedule = 'OFF'
    else:
        pass
    
    return schedule    

In [3]:
# Read master tables

    
with open("/Users/cjyoon/Dropbox/Osler/Osler_shiny/intern/call_schedule_master.yaml") as f:
    my_dict = yaml.safe_load(f)

    
master_schedule = pd.read_excel('/Users/cjyoon/Dropbox/Osler/Osler_shiny/intern/Intern Schedule 2024-2025.xlsx', sheet_name='Schedule')
resident_names = master_schedule.iloc[:, 0].to_list()
series = pd.Series(resident_names)

# Remove NaN values
cleaned_series = series.dropna()

# Convert back to a list
resident_names = cleaned_series.tolist()
print(resident_names)

['Holler_Albert', 'Weinstein_Robert', 'Littman_Emily', 'Rao_Rohit', 'Mathias_Trevor', 'Carter_Julia', 'Reddy_Tina', 'Fatteh_Maria', 'Kim_Minjoo', 'Mollenkopf_Sarah', 'Tracey_Evan', 'Alexander_Mathew', 'Bothwell_James', 'Ermolovich_Jake', 'Payne_Lauren', 'Ancha_Bhavya', 'Kodukula_Sai', 'Yeudall_Scott', 'OPH_Pan-Doh_Nathan', 'OPH_Shah_Rohan', 'Arahirwa_Victor', 'ED_CCU', 'Chabaan_Lara', 'Suresh_Abhilash', 'Riddick_Erica', 'Ye_Congxie', 'Yoon_Christopher', 'Kesaf_Ayse', 'Mhaimeed_Nour', 'Potharazu_Archit', 'Clinton_Ivor', 'Lopez-Silva_Carolina', 'Ozkan_Bige', 'ED_MICU_2', 'Morcos_George', 'Kuzma_Irena', 'ED_MICU', 'Cazimir_Catherine', 'Lopez_Christina', 'Guo_Matthew', 'Bullock_Avery', 'Pazzi_Carlotta', 'Broshkevitch_Anna', 'Lutz_Lydia', 'Rodriguez-Fernandez_Monica', 'Moore_Erica', 'Murphy_Cara', 'Goren_Lea', 'Chang_Michael', 'Jabboure_Fayez', 'Vasoya_Roshan', 'Tarasco_Alexander', 'Koul_Kayshap', 'Khunsriraksakul_Chachrit', 'Zarker_Andrew', 'Ziogos_Timos', 'OPH_Ali_Muhammad', 'OPH_Morello_

In [4]:
def get_schedule(resident_name, master_schedule):
    """gets the specific intern's schedule from the table"""
    filtered_df = master_schedule[master_schedule.iloc[:, 0] == resident_name]
    return (filtered_df.squeeze().to_list()[1:])



In [5]:
new_column_names = [
    f"{col} {master_schedule.iloc[0, idx].strftime('%m/%d/%y')}-{master_schedule.iloc[1, idx].strftime('%m/%d/%y')}"
    if not isinstance(master_schedule.iloc[0, idx], float) and not isinstance(master_schedule.iloc[1, idx], float) 
    else f"{col}"
    for idx, col in enumerate(master_schedule.columns) if not isinstance(col, float)
]

# Update the column names in the DataFrame
master_schedule.columns = new_column_names
print(new_column_names)


master_table_tsv = master_schedule.iloc[3:].fillna('')
master_table_tsv.columns.values[0] = 'name'
master_table_tsv.replace("Subspecialty", "NightWatch", inplace=True)

master_table_tsv.sort_values('name').to_csv('/Users/cjyoon/Dropbox/Osler/Osler_shiny/block_schedule/intern_block_view.tsv', sep='\t', index=False)

['Unnamed: 0', '1A 07/01/24-07/10/24', '1B 07/11/24-07/24/24', '2A 07/25/24-08/07/24', '2B 08/08/24-08/21/24', '3A 08/22/24-09/04/24', '3B 09/05/24-09/18/24', '4A 09/19/24-10/02/24', '4B 10/03/24-10/16/24', '5A 10/17/24-10/30/24', '5B 10/31/24-11/13/24', '6A 11/14/24-11/27/24', '6B 11/28/24-12/11/24', '7A 12/12/24-12/19/24', 'HOLIDAY 1 12/20/24-12/26/24', 'HOLIDAY 2 12/27/24-01/02/25', '7B 01/03/25-01/15/25', '8A 01/16/25-01/29/25', '8B 01/30/25-02/12/25', '9A 02/13/25-02/26/25', '9B 02/27/25-03/12/25', '10A 03/13/25-03/26/25', '10B 03/27/25-04/09/25', '11A 04/10/25-04/23/25', '11B 04/24/25-05/07/25', '12A 05/08/25-05/21/25', '12B 05/22/25-06/04/25', '13A 06/05/25-06/18/25', '13B 06/19/25-06/30/23']


In [6]:
master_table_tsv.sort_values('name')


,name,1A 07/01/24-07/10/24,1B 07/11/24-07/24/24,2A 07/25/24-08/07/24,2B 08/08/24-08/21/24,3A 08/22/24-09/04/24,3B 09/05/24-09/18/24,4A 09/19/24-10/02/24,4B 10/03/24-10/16/24,5A 10/17/24-10/30/24,...,9A 02/13/25-02/26/25,9B 02/27/25-03/12/25,10A 03/13/25-03/26/25,10B 03/27/25-04/09/25,11A 04/10/25-04/23/25,11B 04/24/25-05/07/25,12A 05/08/25-05/21/25,12B 05/22/25-06/04/25,13A 06/05/25-06/18/25,13B 06/19/25-06/30/23
58,,,,2024-07-29 00:00:00,,2024-08-25 00:00:00,,2024-09-22 00:00:00,,2024-10-20 00:00:00,...,,2024-03-09 00:00:00,,2024-04-06 00:00:00,,2024-05-04 00:00:00,,2024-06-01 00:00:00,,2024-06-30 00:00:00
13,Alexander_Mathew,Ambulatory,MICU - F,Thayer - C,Thayer - C,Ambulatory,Brancati - B,Vacation,NightWatch,Ambulatory,...,Ambulatory,MICU - F,Thayer - B,Thayer - B,Ambulatory,Vacation,Thayer - B,Thayer - B,Ambulatory,CCU - B
17,Ancha_Bhavya,MPC - C,Brancati - D,Ambulatory,MICU - D,Barker - D,Barker - D,Ambulatory,Solids,Barker - C,...,Barker - A,Brancati - B,Ambulatory,MPC - C,Barker - D,Barker - D,Ambulatory,NightWatch,MICU - D,Vacation
22,Arahirwa_Victor,Thayer - D,Thayer - D,Ambulatory,Brancati - B,MICU - B,Vacation,Ambulatory,Thayer - B,Thayer - B,...,Thayer - C,MPC - B,Ambulatory,Thayer - C,CCU - A,Vacation,Ambulatory,Thayer - C,Brancati - D,MICU - F
14,Bothwell_James,CCU - A,Ambulatory,Barker - B,Barker - B,MPC - A,Ambulatory,MTL,Vacation,MICU - E,...,Barker - C,Ambulatory,Brancati - C,Barker - A,Barker - A,Ambulatory,MICU - A,Barker - B,CCU - A,Ambulatory
44,Broshkevitch_Anna,Peds,Peds,Peds,MP Clinic,Barker - B,Barker - B,MICU - B,MP Clinic,Peds,...,CCU - A,MP Clinic,Peds,Peds,Peds,Peds,Brancati - B,Vacation,Barker - D,Barker - D
42,Bullock_Avery,Thayer - B,Thayer - B,MICU - A,Ambulatory,Vacation,MTL,Thayer - B,Ambulatory,Brancati - B,...,Thayer - B,Thayer - B,CCU - A,Ambulatory,MPC - B,Thayer - A,Thayer - A,Ambulatory,MICU - C,Vacation
7,Carter_Julia,Ambulatory,Solids,Longcope - D,Longcope - D,Ambulatory,MICU - C,Brancati - D,CCU - C,Ambulatory,...,Ambulatory,Vacation,MICU - D,Longcope - A,Ambulatory,Leuks,Longcope - B,MPC - C,Ambulatory,MICU - B
39,Cazimir_Catherine,Solids,MICU - A,Ambulatory,NightWatch,Janeway - D,Janeway - D,Ambulatory,CCU - A,Janeway - D,...,Janeway - C,Janeway - C,Ambulatory,Janeway - B,MICU - F,Janeway - D,Ambulatory,MTL,MPC - A,Vacation
24,Chabaan_Lara,Liver (MEG),Ambulatory,Longcope - A,Longcope - A,Vacation,Ambulatory,Longcope - C,Longcope - C,MICU - A,...,MICU - D,Ambulatory,Longcope - A,Vacation,CCU - C,Ambulatory,Longcope - D,Longcope - D,MPC - C,Ambulatory


In [10]:
july1_offset = my_dict['july1_offset']
for resident_name in resident_names:
    date_list = []
    schedule_list = []        
    my_blocks = (get_schedule(resident_name, master_schedule))
    print(my_blocks)
    

    calendar = Calendar()
    event = Event()
    with open(f"/Users/cjyoon/Dropbox/Osler/Osler_shiny/intern_schedule/{resident_name}_schedule.tsv", "w") as g:
        g.write('name\tdate\tschedule\n')
        for block_id, rotation in zip(my_dict['block_dates'].keys(), my_blocks):
            # only write out upto holiday block 2 since remaining scheudle has not been finalized yet
#             if block_id in ['7B', '8A', '8B', '9A', '9B', '10A', '10B', '11A', '11B', '12A', '12B', '13A', '13B']:
#             if block_id in ['1A', '1B', '2A', '2B', '3A', '3B', '4A', '4B', '5A', '5B', '6A', '6B', '7A', 'Holiday 1', 'Holiday2']:
            if 1:
        
                if not isinstance(rotation, str):
                    rotation = ' '
                
                rotation = re.sub('\s', '', rotation)

                print(block_id, rotation)

                block_days = block_dates(block_id, my_dict)        

                try:
                    call_cycle = get_specific_rotation_schedule(block_id, rotation, my_dict)
                except:
                    # if specific call cycle is unknown for a rotation, then default to unknown so manual creation can be made
                                        # if specific call cycle is unknown for a rotation, then default to unknown so manual creation can be made
                    if rotation == ' ':
                        call_cycle = [' '] * 14
                    else:
                        call_cycle = ['-'] * 14




                print(f'{block_days} {len(block_days)}')
                print(f'{call_cycle} {len(call_cycle)}')


                if block_id == '1A':
                    # create call cycle that is offset to account for different start date for july1 cycle
                    del call_cycle[:july1_offset]

                # check if block call cycle ends in a post call
                block_days = add_date_postcall_into_next_block(call_cycle, block_days)

                # Loop through each date and corresponding schedule
                for date, schedule in zip(block_days, call_cycle):
                    
                    if rotation in ['MTL', 'Leuks', 'Solids', 'Ambulatory', 'Subspecialty', 'Psych', 'UCM', 'AddictionMedicine']:
                        schedule = oncology_schedule(rotation, date)
                    
                    event = Event()  # Create a new event object inside the loop
                    if rotation == 'Subspecialty': # change Subspecialty into NightWatch more commonly referred.
                        rotation = 'NightWatch'
                    event.name = f'{block_id} {rotation} {schedule}'
                    event.begin = date
                    event.make_all_day()  # Make the event an all-day event
                    print(date, block_id, rotation, schedule)
                    # Add event to calendar
                    calendar.events.add(event)
                    date_string = date.strftime('%m-%d-%Y')
                    g.write(f'{resident_name}\t{date_string}\t{event.name}\n')



        # Write the calendar to a .ics file
        with open(f"/Users/cjyoon/Dropbox/Osler/Osler_shiny/intern_schedule/{resident_name}_schedule.ics", "w") as f:
            f.writelines(calendar.serialize_iter())

    

['Ambulatory', 'MPC - C', 'Barker - D', 'Barker - D', 'Ambulatory', 'MICU - B', 'Barker - A', 'Barker - A', 'Ambulatory', 'Leuks', 'Vacation', 'MICU - D', 'Ambulatory', 'Off', 'Barker - B', 'Barker - C', 'Brancati - D', 'Solids', 'Ambulatory', 'CCU - A', 'Barker - D', 'Barker - D', 'Ambulatory', 'Barker - A', 'Subspecialty', 'Vacation', 'Ambulatory', 'MICU - D']
1A Ambulatory
[datetime.datetime(2024, 7, 1, 0, 0), datetime.datetime(2024, 7, 2, 0, 0), datetime.datetime(2024, 7, 3, 0, 0), datetime.datetime(2024, 7, 4, 0, 0), datetime.datetime(2024, 7, 5, 0, 0), datetime.datetime(2024, 7, 6, 0, 0), datetime.datetime(2024, 7, 7, 0, 0), datetime.datetime(2024, 7, 8, 0, 0), datetime.datetime(2024, 7, 9, 0, 0), datetime.datetime(2024, 7, 10, 0, 0)] 10
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] 14
-
2024-07-01 00:00:00 1A Ambulatory DAY
2024-07-02 00:00:00 1A Ambulatory DAY
2024-07-03 00:00:00 1A Ambulatory DAY
2024-07-04 00:00:00 1A Ambulatory DAY
2024-07-05 00:00:0

['Ambulatory', 'MICU - B', 'Barker - C', 'Barker - C', 'Ambulatory', 'CCU - A', 'Barker - D', 'Barker - D', 'Ambulatory', 'Vacation', 'Leuks', 'MPC - A', 'Ambulatory', 'Off', 'Solids', 'Solids', 'Vacation', 'MICU - C', 'Ambulatory', 'Barker - A', 'MICU - B', 'Brancati - B', 'Ambulatory', 'Subspecialty', 'Barker - A', 'Barker - A', 'Ambulatory', 'Barker - C']
1A Ambulatory
[datetime.datetime(2024, 7, 1, 0, 0), datetime.datetime(2024, 7, 2, 0, 0), datetime.datetime(2024, 7, 3, 0, 0), datetime.datetime(2024, 7, 4, 0, 0), datetime.datetime(2024, 7, 5, 0, 0), datetime.datetime(2024, 7, 6, 0, 0), datetime.datetime(2024, 7, 7, 0, 0), datetime.datetime(2024, 7, 8, 0, 0), datetime.datetime(2024, 7, 9, 0, 0), datetime.datetime(2024, 7, 10, 0, 0)] 10
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] 14
-
2024-07-01 00:00:00 1A Ambulatory DAY
2024-07-02 00:00:00 1A Ambulatory DAY
2024-07-03 00:00:00 1A Ambulatory DAY
2024-07-04 00:00:00 1A Ambulatory DAY
2024-07-05 00:00:00 1A

2025-04-09 00:00:00 10B Psych DAY
11A Barker-C
[datetime.datetime(2025, 4, 10, 0, 0), datetime.datetime(2025, 4, 11, 0, 0), datetime.datetime(2025, 4, 12, 0, 0), datetime.datetime(2025, 4, 13, 0, 0), datetime.datetime(2025, 4, 14, 0, 0), datetime.datetime(2025, 4, 15, 0, 0), datetime.datetime(2025, 4, 16, 0, 0), datetime.datetime(2025, 4, 17, 0, 0), datetime.datetime(2025, 4, 18, 0, 0), datetime.datetime(2025, 4, 19, 0, 0), datetime.datetime(2025, 4, 20, 0, 0), datetime.datetime(2025, 4, 21, 0, 0), datetime.datetime(2025, 4, 22, 0, 0), datetime.datetime(2025, 4, 23, 0, 0)] 14
['DAY', 'OFF', 'CALL', 'POST', 'DAY', 'GOOD', 'CALL', 'POST', 'DAY', 'OFF', 'CALL', 'POST', 'DAY', 'GOOD'] 14
GOOD
2025-04-10 00:00:00 11A Barker-C DAY
2025-04-11 00:00:00 11A Barker-C OFF
2025-04-12 00:00:00 11A Barker-C CALL
2025-04-13 00:00:00 11A Barker-C POST
2025-04-14 00:00:00 11A Barker-C DAY
2025-04-15 00:00:00 11A Barker-C GOOD
2025-04-16 00:00:00 11A Barker-C CALL
2025-04-17 00:00:00 11A Barker-C POST
2

2024-11-29 00:00:00 6B MICU-B GOOD
2024-11-30 00:00:00 6B MICU-B CALL
2024-12-01 00:00:00 6B MICU-B POST
2024-12-02 00:00:00 6B MICU-B OFF
2024-12-03 00:00:00 6B MICU-B CALL
2024-12-04 00:00:00 6B MICU-B POST
2024-12-05 00:00:00 6B MICU-B GOOD
2024-12-06 00:00:00 6B MICU-B CALL
2024-12-07 00:00:00 6B MICU-B POST
2024-12-08 00:00:00 6B MICU-B OFF
2024-12-09 00:00:00 6B MICU-B CALL
2024-12-10 00:00:00 6B MICU-B POST
2024-12-11 00:00:00 6B MICU-B GOOD
7A Longcope-A
[datetime.datetime(2024, 12, 12, 0, 0), datetime.datetime(2024, 12, 13, 0, 0), datetime.datetime(2024, 12, 14, 0, 0), datetime.datetime(2024, 12, 15, 0, 0), datetime.datetime(2024, 12, 16, 0, 0), datetime.datetime(2024, 12, 17, 0, 0), datetime.datetime(2024, 12, 18, 0, 0), datetime.datetime(2024, 12, 19, 0, 0)] 8
['CALL', 'POST', 'DAY', 'OFF', 'CALL', 'POST', 'DAY', 'GOOD'] 8
GOOD
2024-12-12 00:00:00 7A Longcope-A CALL
2024-12-13 00:00:00 7A Longcope-A POST
2024-12-14 00:00:00 7A Longcope-A DAY
2024-12-15 00:00:00 7A Longcope-A

2024-08-01 00:00:00 2A Ambulatory DAY
2024-08-02 00:00:00 2A Ambulatory DAY
2024-08-03 00:00:00 2A Ambulatory OFF
2024-08-04 00:00:00 2A Ambulatory OFF
2024-08-05 00:00:00 2A Ambulatory DAY
2024-08-06 00:00:00 2A Ambulatory DAY
2024-08-07 00:00:00 2A Ambulatory DAY
2B Longcope-B
[datetime.datetime(2024, 8, 8, 0, 0), datetime.datetime(2024, 8, 9, 0, 0), datetime.datetime(2024, 8, 10, 0, 0), datetime.datetime(2024, 8, 11, 0, 0), datetime.datetime(2024, 8, 12, 0, 0), datetime.datetime(2024, 8, 13, 0, 0), datetime.datetime(2024, 8, 14, 0, 0), datetime.datetime(2024, 8, 15, 0, 0), datetime.datetime(2024, 8, 16, 0, 0), datetime.datetime(2024, 8, 17, 0, 0), datetime.datetime(2024, 8, 18, 0, 0), datetime.datetime(2024, 8, 19, 0, 0), datetime.datetime(2024, 8, 20, 0, 0), datetime.datetime(2024, 8, 21, 0, 0)] 14
['POST', 'DAY', 'OFF', 'CALL', 'POST', 'DAY', 'GOOD', 'CALL', 'POST', 'DAY', 'OFF', 'CALL', 'POST', 'DAY'] 14
DAY
2024-08-08 00:00:00 2B Longcope-B POST
2024-08-09 00:00:00 2B Longcope-B

2025-06-14 00:00:00 13A MPC-A CALL
2025-06-15 00:00:00 13A MPC-A POST
2025-06-16 00:00:00 13A MPC-A GOOD
2025-06-17 00:00:00 13A MPC-A CALL
2025-06-18 00:00:00 13A MPC-A POST
13B Vacation
[datetime.datetime(2025, 6, 19, 0, 0), datetime.datetime(2025, 6, 20, 0, 0), datetime.datetime(2025, 6, 21, 0, 0), datetime.datetime(2025, 6, 22, 0, 0), datetime.datetime(2025, 6, 23, 0, 0), datetime.datetime(2025, 6, 24, 0, 0), datetime.datetime(2025, 6, 25, 0, 0), datetime.datetime(2025, 6, 26, 0, 0), datetime.datetime(2025, 6, 27, 0, 0), datetime.datetime(2025, 6, 28, 0, 0), datetime.datetime(2025, 6, 29, 0, 0), datetime.datetime(2025, 6, 30, 0, 0)] 12
['VACATION', 'VACATION', 'VACATION', 'VACATION', 'VACATION', 'VACATION', 'VACATION', 'VACATION', 'VACATION', 'VACATION', 'VACATION', 'VACATION', 'VACATION', 'VACATION'] 14
VACATION
2025-06-19 00:00:00 13B Vacation VACATION
2025-06-20 00:00:00 13B Vacation VACATION
2025-06-21 00:00:00 13B Vacation VACATION
2025-06-22 00:00:00 13B Vacation VACATION
202

2025-03-24 00:00:00 10A MPC-A GOOD
2025-03-25 00:00:00 10A MPC-A CALL
2025-03-26 00:00:00 10A MPC-A POST
10B Ambulatory
[datetime.datetime(2025, 3, 27, 0, 0), datetime.datetime(2025, 3, 28, 0, 0), datetime.datetime(2025, 3, 29, 0, 0), datetime.datetime(2025, 3, 30, 0, 0), datetime.datetime(2025, 3, 31, 0, 0), datetime.datetime(2025, 4, 1, 0, 0), datetime.datetime(2025, 4, 2, 0, 0), datetime.datetime(2025, 4, 3, 0, 0), datetime.datetime(2025, 4, 4, 0, 0), datetime.datetime(2025, 4, 5, 0, 0), datetime.datetime(2025, 4, 6, 0, 0), datetime.datetime(2025, 4, 7, 0, 0), datetime.datetime(2025, 4, 8, 0, 0), datetime.datetime(2025, 4, 9, 0, 0)] 14
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] 14
-
2025-03-27 00:00:00 10B Ambulatory DAY
2025-03-28 00:00:00 10B Ambulatory DAY
2025-03-29 00:00:00 10B Ambulatory OFF
2025-03-30 00:00:00 10B Ambulatory OFF
2025-03-31 00:00:00 10B Ambulatory DAY
2025-04-01 00:00:00 10B Ambulatory DAY
2025-04-02 00:00:00 10B Ambulatory DAY
2025

2024-10-25 00:00:00 5A CCU-C GOOD
2024-10-26 00:00:00 5A CCU-C CALL
2024-10-27 00:00:00 5A CCU-C POST
2024-10-28 00:00:00 5A CCU-C OFF
2024-10-29 00:00:00 5A CCU-C CALL
2024-10-30 00:00:00 5A CCU-C POST
5B Ambulatory
[datetime.datetime(2024, 10, 31, 0, 0), datetime.datetime(2024, 11, 1, 0, 0), datetime.datetime(2024, 11, 2, 0, 0), datetime.datetime(2024, 11, 3, 0, 0), datetime.datetime(2024, 11, 4, 0, 0), datetime.datetime(2024, 11, 5, 0, 0), datetime.datetime(2024, 11, 6, 0, 0), datetime.datetime(2024, 11, 7, 0, 0), datetime.datetime(2024, 11, 8, 0, 0), datetime.datetime(2024, 11, 9, 0, 0), datetime.datetime(2024, 11, 10, 0, 0), datetime.datetime(2024, 11, 11, 0, 0), datetime.datetime(2024, 11, 12, 0, 0), datetime.datetime(2024, 11, 13, 0, 0)] 14
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] 14
-
2024-10-31 00:00:00 5B Ambulatory DAY
2024-11-01 00:00:00 5B Ambulatory DAY
2024-11-02 00:00:00 5B Ambulatory OFF
2024-11-03 00:00:00 5B Ambulatory OFF
2024-11-04 00:

In [8]:
print(resident_name, block_id, rotation)

ED_CCU_2 13B 


In [9]:
def authenticate_google_drive():
    creds = None

    # Use token.json to store the user's access and refresh tokens
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    # If there are no valid credentials, let the user log in and authorize the app
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Use the client_secret.json to start the OAuth flow
            flow = InstalledAppFlow.from_client_secrets_file(
                '/Users/cjyoon/Dropbox/Osler/osler/client_secret_257342927366-4q3itidpqit3vcehjca5rvlg9m2evhsi.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        # Save the credentials for future runs in token.json
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    # Build the Google Drive service object
    service = build('drive', 'v3', credentials=creds)
    return service



def upload_ics_to_drive(file_path):
    service = authenticate_google_drive()

    # Upload the file to Google Drive with the correct MIME type for .ics files
    file_metadata = {'name': os.path.basename(file_path), 'mimeType': 'text/calendar'}
    media = MediaFileUpload(file_path, mimetype='text/calendar', parent='osler_schedule' )
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    # Make the file public and get the shareable link
    file_id = file.get('id')
    service.permissions().create(fileId=file_id, body={'type': 'anyone', 'role': 'reader'}).execute()
    shareable_url = f"https://drive.google.com/file/d/osler_schedule/{file_id}/view"
    
    return shareable_url


# for resident in resident_names:
#     file_path = f'/Users/cjyoon/Dropbox/Osler/osler/data/{resident}_schedule.ics'
#     shareable_link = upload_ics_to_drive(file_path)
#     print(f"{resident} schedule: {shareable_link}")